# Lesson 4: Building a Multi-Document Agent

## Setup

In [1]:
import nest_asyncio
nest_asyncio.apply()

## 1. Setup an agent over 3 papers

**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [4]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]

In [5]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: selfrag.pdf


In [9]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
print(initial_tools)

[<llama_index.core.tools.function_tool.FunctionTool object at 0x000001EC47668760>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x000001EC47669BD0>, <llama_index.core.tools.function_tool.FunctionTool object at 0x000001EC47C00A00>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x000001EC47C02DD0>, <llama_index.core.tools.function_tool.FunctionTool object at 0x000001EC476CBCD0>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x000001EC476C8FA0>]


In [10]:
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

from dotenv import load_dotenv
import os

load_dotenv("../../.env")

token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    token=token,
)

C:\Users\Ghost\AppData\Local\Temp\ipykernel_16388\4153170195.py:10: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  llm = HuggingFaceInferenceAPI(


In [13]:
from llama_index.llms.gemini import Gemini
llm = Gemini()

In [14]:
len(initial_tools)

6

In [16]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [10]:
response = agent.query(
    "Tell me about the evaluation dataset used in LongLoRA, "
    "and then tell me about the evaluation results"
)

Added user message to memory: Tell me about the evaluation dataset used in LongLoRA, and then tell me about the evaluation results
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation dataset"}
=== Function Output ===
PG19
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation results"}
=== Function Output ===
The evaluation results show that the model achieves reasonable passkey retrieval accuracy until around 33k to 34k context length. By extending the position interpolation to 48k, the model can handle longer documents, presenting moderate retrieval ability (60%-90% accuracy) in the range of 33k to 45k. However, there is a sharp accuracy degradation observed after the 4k context length for the Llama2 7B model, even with the position interpolation extended.
=== LLM Response ===
The evaluation dataset used in LongLoRA is not explicitly mentioned in the paper. However, the evaluation results show that

In [11]:
response = agent.query("Give me a summary of both Self-RAG and LongLoRA")
print(str(response))

Added user message to memory: Give me a summary of both Self-RAG and LongLoRA
=== Calling Function ===
Calling function: summary_tool_selfrag with args: {"input": "Self-RAG"}
=== Function Output ===
Self-RAG is a framework that enhances the quality and factuality of a large language model through retrieval and self-reflection. It involves training a single arbitrary language model to adaptively retrieve passages on-demand, generate and reflect on retrieved passages and its own generations using special tokens called reflection tokens. This framework enables the language model to tailor its behavior to diverse task requirements during the inference phase, significantly outperforming state-of-the-art language models and retrieval-augmented models on various tasks by improving factuality and citation accuracy for long-form generations.
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "LongLoRA"}
=== Function Output ===
LongLoRA is an efficient method f

## 2. Setup an agent over 11 papers

### Download 11 ICLR papers

In [12]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "loftq.pdf",
    "swebench.pdf",
    "selfrag.pdf",
    "zipformer.pdf",
    "values.pdf",
    "finetune_fair_diffusion.pdf",
    "knowledge_card.pdf",
    "metra.pdf",
    "vr_mcl.pdf"
]

To download these papers, below is the needed code:


    #for url, paper in zip(urls, papers):
         #!wget "{url}" -O "{paper}"
    
    
**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [13]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: loftq.pdf
Getting tools for paper: swebench.pdf
Getting tools for paper: selfrag.pdf
Getting tools for paper: zipformer.pdf
Getting tools for paper: values.pdf
Getting tools for paper: finetune_fair_diffusion.pdf
Getting tools for paper: knowledge_card.pdf
Getting tools for paper: metra.pdf
Getting tools for paper: vr_mcl.pdf


### Extend the Agent with Tool Retrieval

In [14]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [15]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [17]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [18]:
tools = obj_retriever.retrieve(
    "Tell me about the eval dataset used in MetaGPT and SWE-Bench"
)

In [22]:
tools[2].metadata

ToolMetadata(description='Useful for summarization questions related to swebench', name='summary_tool_swebench', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [23]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [24]:
response = agent.query(
    "Tell me about the evaluation dataset used "
    "in MetaGPT and compare it against SWE-Bench"
)
print(str(response))

Added user message to memory: Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation dataset used in MetaGPT"}
=== Function Output ===
The evaluation dataset used in MetaGPT includes HumanEval, MBPP, and SoftwareDev.
=== Calling Function ===
Calling function: summary_tool_swebench with args: {"input": "evaluation dataset used in SWE-Bench"}
=== Function Output ===
The evaluation dataset used in SWE-Bench consists of task instances collected from real GitHub issues and pull requests across popular Python repositories. It includes task instructions, issue text, retrieved files and documentation, example patch files, and prompts for generating patch files. Models like ChatGPT-3.5, GPT-4, Claude 2, and SWE-Llama were evaluated based on their ability to resolve issues, with Claude 2 being the best performer. The dataset is designed to be challenging for evaluating 

In [25]:
response = agent.query(
    "Compare and contrast the LoRA papers (LongLoRA, LoftQ). "
    "Analyze the approach in each paper first. "
)

Added user message to memory: Compare and contrast the LoRA papers (LongLoRA, LoftQ). Analyze the approach in each paper first. 
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "Compare and contrast the approach in the LongLoRA paper."}
=== Function Output ===
The approach in the LongLoRA paper introduces an efficient fine-tuning method that focuses on extending the context sizes of pre-trained large language models (LLMs) while minimizing computational costs. It utilizes Shifted Sparse Attention (S2-Attn) during training to approximate long contexts efficiently, while retaining the original attention architecture during inference. This method combines S2-Attn with Low-rank Adaptation (LoRA) to achieve strong empirical results on various tasks and context lengths, showcasing improved training speed and memory efficiency while maintaining performance. In contrast, traditional Low-rank Adaptation (LoRA) alone is found to be less effective and efficie